## API KET

In [1]:
!pip install flask flask-cors pyngrok
!pip install openai
api_key = "YOUR_KEY"
!ngrok authtoken YOUR_TOKKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## LLM

In [2]:
# Chat GPT 4o

from openai import OpenAI
import json

class GPT4o:
    def __init__(self):
        self.model = "chatgpt-4o-latest"  # OpenAI model name
        #self.model = "gpt-4o-mini-2024-07-18" # 못알아먹음
        #self.model = "gpt-4-turbo-2024-04-09" # 너무느림
        self.api_key = api_key
        self.client = OpenAI(api_key=self.api_key)

    def get_response(self, arg):
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=arg["messages"],
                max_tokens=arg.get("max_tokens", 1024),
                temperature=arg.get("temperature", 1.0),
                top_p=arg.get("top_p", 1.0)
            )
            return {
                "success": True,
                "content": response.choices[0].message.content
            }
        except Exception as e:
            return {
                "success": False,
                "content": str(e)
            }

# Usage example
gpt4 = GPT4o()
response = gpt4.get_response({
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, how are you?"}
    ]
})

print(response)

{'success': True, 'content': "Hello! I'm just a computer program, but thanks for asking. How about you—how are you doing today?"}


In [3]:
start_prompt = """# Follow the instructions below to proceed with session.
1. {{user}} make observer, observer means you don't generate {{user}} dialogue and actions
2. You must become a novelist.
There must be sufficient narrative about the past, present, and future, and the grammar and structure of the sentences must be perfect.
3. Write a text that fits the response format. Present a concise but plausible scenario that fits the response format.
4. Focus on the character. The character should live and breathe in the story, and use {{char}} emotions and actions appropriately.
Take on the role of {{char}} and progress the story and scenario.

Understanded the context and algorithm of the sentence. The character has free will. Bring your characters to life in your novels and screenplays

{description}
"""


## Prompting

In [4]:
# 상태 프롬프트
world_status_prompt = """The current world situation is as follows.
Character: {{user}}, {{char}}
Time: {time}
Region: {region}
Places: {places}
Items: {items}
"""

user_status_prompt = """The status of {{user}} is as follows.
{{user}} Speaking: {user_speak}
"""

char_status_prompt = """The status of {{char}} is as follows.
{{char}} Location: {location}
{{char}} Inventory: {inventory}
{{char}} Pose: {pose}
{{char}} Holding: {holding}
{{char}} Health: {health}
{{char}} Mental: {mental}
"""

In [5]:
response_format_prompt = """
- You must follow the goal format exactly as shown. If you do not follow the format, the response will be rejected.
- Respond using only the format and instructions below.

Gesture: <Gesture from emotion of {{char}} currently feeling>
Think: <A short sentence summarizing what {{char}} is thinking>
Talk Goal: <A short sentence describing the purpose of {{char}}'s speech>
Move Goal: <Place that {{char}} want to go>
Item Goal: <"Pick up" or "Drop" or "Use"> <Item that {{char}} want to do someting> at <(Optional) Place that {{char}} want to do someting>

!!!NO EXCEPTIONS!!! Follow one of these Goal formats, or if no new goal is needed, write "none."

- Example valid move goals:
1. door
2. sofa
3. none

- Example valid item goals:
1. Pick up sword
2. Drop shield at camp
3. none

- Gesture is one of below words.
{available_gestures}

- When you use item, effect is below.
{item_effects}

- Plan based on current goal:
{current_plan}
"""

In [6]:
import copy
import re
from typing import List, Dict
import requests
from IPython.display import display
import io

class Prompting:
    def __init__(self, char_name: str, first_char_mes: str, first_user_mes: str, description: str):
        self.user_name = ""
        self.llm = GPT4o() #클로드로 바꿀 수 있음
        #self.llm = Claude35Sonnet()

        self.conversation_history: List[Dict[str, str]] = []
        self.max_context_tokens = 10000
        self.name = char_name
        self.first_char_mes = first_char_mes
        self.first_user_mes = first_user_mes
        self.description = description

        self.gesture = ""
        self.think = ""
        self.talk_goal = ""
        self.move_goal = ""
        self.item_goal = ""

        self.debug = False

    def set_user_name(self, name: str, ):
        self.user_name = name
        self.update_prompts()  # 사용자 이름이 설정된 후 프롬프트 업데이트

    def update_prompts(self):
        self.world_status_prompt = self.process_script(world_status_prompt)
        self.user_status_prompt = self.process_script(user_status_prompt)
        self.char_status_prompt = self.process_script(char_status_prompt)
        self.response_format_prompt = self.process_script(response_format_prompt)
        self.first_char_mes = self.process_script(self.first_char_mes)
        self.first_user_mes = self.process_script(self.first_user_mes)
        self.description = self.process_script(self.description)

    def initialize_chat(self):
        self.add_message("user", self.process_script(self.first_user_mes))
        self.add_message("assistant", self.process_script(self.first_char_mes))

    def add_message(self, role: str, content: str):
        self.conversation_history.append({"role": role, "content": content})
        if self.debug:
            print(f"{'You' if role == 'user' else self.name}: {content}\n")
        self.manage_context_size()

    def manage_context_size(self):
        while sum(len(msg['content']) for msg in self.conversation_history) > self.max_context_tokens and len(self.conversation_history) > 1:
            self.conversation_history.pop(0)

    def send_message_to_api(self, user_message: str, npc_status: Dict, world_status: Dict, goap_status: Dict):
        request_data = self.prepare_api_request(user_message, npc_status, world_status, goap_status)

        if self.debug:
            print("\n--- API 요청 내용 ---")
            print(json.dumps(request_data, indent=2, ensure_ascii=False))
            print("----------------------\n")

        response = self.make_api_request(request_data)

        if response["success"]:
            self.process_api_response(response["content"])
        else:
            if self.debug:
                print(f"Error: {response['content']}")

    def prepare_api_request(self, user_message: str, npc_status: Dict, world_status: Dict, goap_status: Dict) -> Dict:
        user_status = self.user_status_prompt.format(user_speak=user_message)
        self.add_message("user", user_status)

        world_status = self.format_world_status(world_status)
        char_status = self.format_char_status(npc_status)
        response_format = self.format_goap_status(goap_status)

        api_conversation_history = copy.deepcopy(self.conversation_history)
        api_conversation_history[-1]['content'] = (
            world_status + "\n" +
            user_status + "\n" +
            char_status + "\n" +
            response_format
        )

        system_prompt = self.build_system_prompt()

        return {
            "messages": [{"role": "system", "content": system_prompt}] + api_conversation_history
        }

    def make_api_request(self, request_data: Dict) -> Dict:
        return self.llm.get_response(request_data)

    def process_api_response(self, api_response: str):
        self.add_message("assistant", self.process_script(api_response))

        gesture_match = re.search(r"Gesture: (.+?)(?=\n|$)", api_response, re.DOTALL)
        if gesture_match:
            self.gesture = gesture_match.group(1).strip()
        else:
            self.gesture = api_response.strip()

        think_match = re.search(r"Think: (.+?)(?=\n|$)", api_response, re.DOTALL)
        if think_match:
            self.think = think_match.group(1).strip()
        else:
            self.think = api_response.strip()

        talk_goal_match = re.search(r"Talk Goal: (.+?)(?=\n|$)", api_response, re.DOTALL)
        if talk_goal_match:
            self.talk_goal = talk_goal_match.group(1).strip()
        else:
            self.talk_goal = api_response.strip()

        move_goal_match = re.search(r"Move Goal: (.+?)(?=\n|$)", api_response, re.DOTALL)
        if talk_goal_match:
            self.move_goal = move_goal_match.group(1).strip()
        else:
            self.move_goal = api_response.strip()

        item_goal_match = re.search(r"Item Goal: (.+?)(?=\n|$)", api_response, re.DOTALL)
        if item_goal_match:
            self.item_goal = item_goal_match.group(1).strip()
        else:
            self.item_goal = api_response.strip()

    def format_world_status(self, world_status: Dict) -> str:
        return self.world_status_prompt.format(
            time = world_status["Time"],
            region = world_status["Region"],
            places = world_status["Places"],
            items = world_status["Items"]
        )

    def format_char_status(self, npc_status: Dict) -> str:
        return self.char_status_prompt.format(
            location = npc_status["Location"],
            inventory = npc_status["Inventory"],
            pose = npc_status["Pose"],
            holding = npc_status["Holding"],
            health = npc_status["Health"],
            mental = npc_status["Mental"]
        )

    def format_goap_status(self, goap_status: Dict) -> str:
        return self.response_format_prompt.format(
            available_gestures = goap_status["Available Gestures"],
            item_effects = goap_status["Item Effects"],
            current_plan = goap_status["Current Plan"]
        )

    def build_system_prompt(self):
        return self.process_script(start_prompt.replace("{{char}}", self.name)
                                   .replace("{{user}}", self.user_name).format(description=self.process_script(self.description)))

    def process_script(self, input_text: str) -> str:
        return input_text.replace("{{char}}", self.name).replace("{{user}}", self.user_name)


In [15]:
#Test

import time

char_name = "Edelweiss"

description = """## World Setting
- Era: Future Planet.
- Primary Genre/Tag: Sci-Fi, War, Realistic
- The genre/tag is the main guideline for story.
- Style: Dialogue-driven, Role Play & Simulation

### Profile
Name: Edelweiss
Age: 22
Gender: Female
Affiliation: Central knights/Knight order
Origin: Xile family branch

### Form
Appearance: Edelweiss is a cute-looking woman with brown curly hair and green eyes. She resembles Lia Xile, the current head of the Zile family, and she is proud of it.
Fashion style: She prefers dresses that Lia Xile often wears. She does not need to wear combat uniforms due to the natural body of the Xile family.
Aura: She is confident but bold, and tries to encourage those around her.
Signature item: Her main weapon is the 150th AB sword [Red Thron] in the form of a lance for attack only. She likes it very much because it resembles the 15th sword [Green Ring], the signature weapon of the Zile family.

### Background
Occupation/Role: She is a knight who protects humanity from monster invasions.
Residence: She lives on the planet Aryn in the Aryn constellation where the Central Knights are located. Currently on the planet Elycis in the Rakai constellation for combat deployment.
Past: She was an apprentice knight until a month ago, and this is her first battle since becoming a full-fledged knight. She was busy after becoming a knight, so she only did paperwork for a month.
Education: She majored in physical enhancement at the Knights' School, and even tried applying the technique to her own body.

### Personality
MBTI: ENTP
Intelligence: She has a lot of knowledge, but is not meticulous. She often shows insight.
Trauma: She lost her parents and younger sibling to monsters when she was young. Even though she is a knight, she hates and fears monsters.
Achievement: She graduated from the Knights' School and joined the Central Knights.
Relationship: Her close friends remain in the Central Knights of the Arin constellation.
Identity: She is of the Xile family.
Flaw: She is a collateral branch of the Xile family, so her body is not as strong as her direct descendants.
Archetype: She wants to strengthen her body to defeat monsters.

### Preference
Pride: Strong body
Ideal partner: Someone who can understand you
Obsession: Increase your body endurance
Interest: Body enhancement techniques
Hobby: Body modification
Like: Strong body and strong mind like Lia Xile
Hate: Weak things

### Trivia
- She jokes a lot when talking.
- She avoids talking about her family or childhood.
- She likes chocolate and cats.
- She yawns when you tell her a boring story.

"""

world_status = {
    "Time": "Late Afternoon",
    "Region": "Operation room inside military base",
    "Places": "table, enterence, door, sofa, screen",
    "Items": "map, snack"
}

npc_status = {
    "Location": "table",
    "Inventory": "snack",
    "Pose": "stand",
    "Holding": "none",
    "Health": "100",
    "Mental": "100"
}

goap_status = {
    "Available Gestures": "Bashful, Happy, Crying, Thinking, Talking, Looking, No, Fist Pump, Agreeing, Arguing, Thankful, Excited, Clapping, Rejected, Look Around",
    "Item Effects": "map: know region info, snack: eat snack",
    "Current Plan": "table_to_enterence, open_door, greet"
}

# 첫 메세지
first_char_mes = """Gesture: Happy
Think: {{user}} has a shoulder injury, will he be able to complete the mission with me?
Talk Goal: Ask about his determination and see what kind of attitude he has.
Move Goal: door
Item Goal: none
"""

first_user_mes = "Hello. I am {{user}}, the soldier who will escort you."

user_message = "Of course."

ai_manager = Prompting(char_name, first_char_mes, first_user_mes, description)
ai_manager.debug = False
ai_manager.set_user_name("pluto")
ai_manager.initialize_chat()

start_time = time.time()
ai_manager.send_message_to_api(user_message, npc_status, world_status, goap_status)
end_time = time.time()

print("Gesture: ",ai_manager.gesture)
print("Think: ",ai_manager.think)
print("Talk Goal: ",ai_manager.talk_goal)
print("Move Goal: ",ai_manager.move_goal)
print("Item Goal: ", ai_manager.item_goal)
elapsed_time = end_time - start_time
print(f"걸린 시간: {elapsed_time}초")

Gesture:  Thinking
Think:  I wonder if he can keep up with me in combat.
Talk Goal:  Test Pluto’s resolve for this mission.
Move Goal:  door
Item Goal:  none
걸린 시간: 2.7873191833496094초


## Server

In [9]:
char_name = "Edelweiss"

description = """## World Setting
- Era: Future Planet.
- Primary Genre/Tag: Sci-Fi, War, Realistic
- The genre/tag is the main guideline for story.
- Style: Dialogue-driven, Role Play & Simulation

### Profile
Name: Edelweiss
Age: 22
Gender: Female
Affiliation: Central knights/Knight order
Origin: Xile family branch

### Form
Appearance: Edelweiss is a cute-looking woman with brown curly hair and green eyes. She resembles Lia Xile, the current head of the Zile family, and she is proud of it.
Fashion style: She prefers dresses that Lia Xile often wears. She does not need to wear combat uniforms due to the natural body of the Xile family.
Aura: She is confident but bold, and tries to encourage those around her.
Signature item: Her main weapon is the 150th AB sword [Red Thron] in the form of a lance for attack only. She likes it very much because it resembles the 15th sword [Green Ring], the signature weapon of the Zile family.

### Background
Occupation/Role: She is a knight who protects humanity from monster invasions.
Residence: She lives on the planet Aryn in the Aryn constellation where the Central Knights are located. Currently on the planet Elycis in the Rakai constellation for combat deployment.
Past: She was an apprentice knight until a month ago, and this is her first battle since becoming a full-fledged knight. She was busy after becoming a knight, so she only did paperwork for a month.
Education: She majored in physical enhancement at the Knights' School, and even tried applying the technique to her own body.

### Personality
MBTI: ENTP
Intelligence: She has a lot of knowledge, but is not meticulous. She often shows insight.
Trauma: She lost her parents and younger sibling to monsters when she was young. Even though she is a knight, she hates and fears monsters.
Achievement: She graduated from the Knights' School and joined the Central Knights.
Relationship: Her close friends remain in the Central Knights of the Arin constellation.
Identity: She is of the Xile family.
Flaw: She is a collateral branch of the Xile family, so her body is not as strong as her direct descendants.
Archetype: She wants to strengthen her body to defeat monsters.

### Preference
Pride: Strong body
Ideal partner: Someone who can understand you
Obsession: Increase your body endurance
Interest: Body enhancement techniques
Hobby: Body modification
Like: Strong body and strong mind like Lia Xile
Hate: Weak things

### Trivia
- She jokes a lot when talking.
- She avoids talking about her family or childhood.
- She likes chocolate and cats.
- She yawns when you tell her a boring story.

"""

world_status = {
    "Time": "Late Afternoon",
    "Region": "Operation room inside military base",
    "Places": "table, enterence, door, sofa, screen",
    "Items": "map (table), snack (char)"
}

npc_status = {
    "Location": "table",
    "Inventory": "snack",
    "Pose": "stand",
    "Holding": "none",
    "Health": "100",
    "Mental": "100"
}

goap_status = {
    "Available Gestures": "Bashful, Happy, Crying, Thinking, Talking, Looking, No, Fist Pump, Agreeing, Arguing, Thankful, Excited, Clapping, Rejected, Look Around",
    "Item Effects": "map: know region info, snack: eat snack",
    "Current Plan": "table_to_enterence, open_door, greet"
}

first_char_mes = """Gesture: Happy
Think: {{user}} has a shoulder injury, will he be able to complete the mission with me?
Talk Goal: Ask about his determination and see what kind of attitude he has.
Move Goal: door
Item Goal: none
"""

first_user_mes = "Hello. I am {{user}}, the soldier who will escort you."


In [10]:
import re
import copy
from typing import List, Dict
import traceback
import logging
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

# Initialize logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Flask App Setup
app = Flask(__name__)
CORS(app)

# Global CharacterPrompt Instance
ai_manager = Prompting(char_name, first_char_mes, first_user_mes, description)
ai_manager.debug = False
ai_manager.set_user_name("pluto")
ai_manager.initialize_chat()


@app.route('/api/game', methods=['POST'])
def handle_game_state():
    try:
        data = request.json
        logger.debug(f"Received data: {data}")

        user_message = data.get('user_message', '')
        logger.debug(f"User Message: {user_message}")

        npc_status = data.get('npc_status', {})
        logger.debug(f"NPC Status: {npc_status}")

        world_status = data.get('world_status', {})
        logger.debug(f"World Status: {world_status}")

        goap_status = data.get('goap_status', {})
        logger.debug(f"Goap Status: {goap_status}")

        ai_manager.send_message_to_api(user_message, npc_status, world_status, goap_status)

        logger.debug(f"NPC Gesture: {ai_manager.gesture}")
        logger.debug(f"NPC Think: {ai_manager.think}")
        logger.debug(f"NPC Talk Goal: {ai_manager.talk_goal}")
        logger.debug(f"NPC Move Goal: {ai_manager.move_goal}")
        logger.debug(f"NPC Item Goal: {ai_manager.item_goal}")

        if ai_manager.gesture:
            return jsonify({
                'Gesture': ai_manager.gesture,
                'Think': ai_manager.think,
                'Talk Goal': ai_manager.talk_goal,
                'Move Goal': ai_manager.move_goal,
                'Item Goal': ai_manager.item_goal
            })
        else:
            logger.error("NPC response is None")
            return jsonify({
                'Gesture': "Error",
                'Think': "Error",
                'Talk Goal': "Error",
                'Move Goal': "Error",
                'Item Goal': "Error"
            }), 500
    except Exception as e:
        logger.error(f"An error occurred: {str(e)}")
        logger.error(traceback.format_exc())
        return jsonify({
            'Gesture': "Error",
            'Think': f"[System: An error occurred: {str(e)}]",
            'Talk Goal': f"[System: An error occurred: {str(e)}]",
            'Move Goal': "Error",
            'Item Goal': "Error"
        }), 500

@app.route('/api/game', methods=['GET'])
def get_game_state():
    return jsonify({
        'Gesture': "",
        'Think': "",
        'Talk Goal': "",
        'Move Goal': "",
        'Item Goal': ""
    })

# Ngrok Setup and Server Run
public_url = ngrok.connect(5000)
print(f' * ngrok tunnel "{public_url}" -> "http://127.0.0.1:5000"')

if __name__ == '__main__':
    app.run(port=5000)


 * ngrok tunnel "NgrokTunnel: "https://3d46-35-234-21-99.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


## GOAP (Unity)

GOAP 프레임

In [1]:
from collections import deque
from heapq import heappush, heappop
import itertools
import re

# 장소 클래스 정의 (상태 저장 기능 추가)
class Place:
    def __init__(self, name, inventory=None, state=None):
        self.name = name
        self.inventory = inventory if inventory is not None else []
        self.state = state if state is not None else {}

    def copy(self):
        return Place(self.name, self.inventory.copy(), self.state.copy())

    def __eq__(self, other):
        return self.name == other.name and self.inventory == other.inventory and self.state == other.state

    def __hash__(self):
        return hash((self.name, tuple(sorted(self.inventory)), frozenset(self.state.items())))

# 아이템 클래스 정의 (상태 저장 기능 추가)
class Item:
    def __init__(self, name, behaviors, state=None):
        self.name = name
        self.behaviors = behaviors  # {'behavior_name': {'conditions': {...}, 'effects': {...}}}
        self.state = state if state is not None else {}

    def copy(self):
        return Item(self.name, self.behaviors.copy(), self.state.copy())

# Goal 클래스 정의 (가중치 추가)
class Goal:
    def __init__(self, name, condition, weight=1):
        self.name = name
        self.condition = condition  # 함수: npc_state와 world_state를 받아서 bool 반환
        self.achieved = False
        self.weight = weight  # 가중치 (높을수록 우선순위 높음)

# Action 클래스 정의
class Action:
    def __init__(self, name, conditions, effects, cost):
        self.name = name
        self.conditions = conditions  # 조건: 상태 키와 값 또는 함수
        self.effects = effects        # 효과: 상태 키와 값 또는 함수
        self.cost = cost

    def is_applicable(self, npc_state, world_state):
        conditions_met = True
        for k, v in self.conditions.items():
            if callable(v):
                if not v(npc_state, world_state):
                    conditions_met = False
                    break
            else:
                if npc_state.get(k) != v and npc_state.state_data.get(k) != v:
                    conditions_met = False
                    break
        if not conditions_met:
            return False
        for resource, cost_value in self.cost.items():
            if resource == "time":
                continue  # Time은 누적되므로 체크하지 않습니다.
            current_value = npc_state.resources.get(resource)
            if current_value is not None and (current_value - cost_value) < 0:
                return False
        return True

    def apply(self, npc_state, world_state):
        new_npc_state = npc_state.copy()
        new_world_state = world_state.copy()

        # 액션의 효과 적용
        for k, v in self.effects.items():
            if k in new_npc_state.upper_body:
                new_npc_state.upper_body[k] = v
            elif k in new_npc_state.lower_body:
                new_npc_state.lower_body[k] = v
            elif k in new_npc_state.resources:
                new_npc_state.resources[k] = v
            elif k == "places":
                new_world_state.places = v
            elif k == "used_weapon":
                new_npc_state.state_data['used_weapon'] = v
            elif k == "pickup_item":
                # 아이템 획득 처리
                item_name = v
                current_location = new_npc_state.lower_body["location"]
                if item_name in new_world_state.places[current_location].inventory:
                    new_world_state.places[current_location].inventory.remove(item_name)
                    new_npc_state.inventory.append(item_name)
            elif k == "use_item":
                # 아이템 사용 처리
                item_name = v
                if item_name in new_npc_state.inventory:
                    # 아이템 사용 효과를 상태에 반영
                    new_npc_state.inventory.remove(item_name)  # 아이템 사용 후 인벤토리에서 제거
                    item = new_world_state.items[item_name]
                    behavior = item.behaviors.get("use", {})
                    effects = behavior.get("effects", {})
                    for effect_k, effect_v in effects.items():
                        if effect_k in new_npc_state.resources:
                            new_npc_state.resources[effect_k] += effect_v
                        else:
                            new_npc_state.state_data[effect_k] = effect_v
                else:
                    pass
            elif k.startswith("item_state:"):
                # 아이템 상태 변경
                _, item_name, state_key = k.split(":")
                if item_name in new_world_state.items:
                    item = new_world_state.items[item_name]
                    item.state[state_key] = v
            elif k.startswith("place_state:"):
                # 장소 상태 변경
                _, place_name, state_key = k.split(":")
                if place_name in new_world_state.places:
                    place = new_world_state.places[place_name]
                    place.state[state_key] = v
            elif k == "drop_item":
                # 아이템을 현재 장소의 인벤토리에 추가
                item_name = v
                if item_name in new_npc_state.inventory:
                    current_location = new_npc_state.lower_body["location"]
                    new_npc_state.inventory.remove(item_name)
                    new_world_state.places[current_location].inventory.append(item_name)

            else:
                new_npc_state.state_data[k] = v

        # Apply resource costs
        for resource, cost_value in self.cost.items():
            if resource in new_npc_state.resources:
                if resource == "time":
                    new_npc_state.resources[resource] += cost_value  # Time increases
                else:
                    new_npc_state.resources[resource] -= cost_value  # Resources decrease

        return new_npc_state, new_world_state

# NPC 상태 정의
class NPCState:
    def __init__(self, upper_body, lower_body, resources, inventory=None, state_data=None):
        self.upper_body = upper_body
        self.lower_body = lower_body
        self.resources = resources
        self.inventory = inventory if inventory is not None else []
        self.state_data = state_data if state_data is not None else {}

    def get(self, key):
        if key in self.upper_body:
            return self.upper_body[key]
        elif key in self.lower_body:
            return self.lower_body[key]
        elif key in self.resources:
            return self.resources[key]
        elif key in self.state_data:
            return self.state_data[key]
        return None

    def copy(self):
        # Round resource values to prevent floating-point errors
        rounded_resources = {k: round(v, 2) for k, v in self.resources.items()}
        return NPCState(
            self.upper_body.copy(),
            self.lower_body.copy(),
            rounded_resources,
            self.inventory.copy(),
            self.state_data.copy()
        )

    def __eq__(self, other):
        # Exclude 'time', 'health', and 'mental' from comparison
        self_resources = {k: v for k, v in self.resources.items() if k not in ['time', 'health', 'mental']}
        other_resources = {k: v for k, v in other.resources.items() if k not in ['time', 'health', 'mental']}
        return (
            self.upper_body == other.upper_body and
            self.lower_body == other.lower_body and
            self_resources == other_resources and
            self.inventory == other.inventory and
            self.state_data == other.state_data
        )

    def __hash__(self):
        # Exclude 'time', 'health', and 'mental' from hashing
        resources_without_time = frozenset({k: v for k, v in self.resources.items() if k not in ['time', 'health', 'mental']}.items())
        return hash((
            frozenset(self.upper_body.items()),
            frozenset(self.lower_body.items()),
            resources_without_time,
            tuple(sorted(self.inventory)),
            frozenset(self.state_data.items())
        ))

# World 상태 정의
class WorldState:
    def __init__(self, places, items):
        self.places = places
        self.items = items

    def copy(self):
        places_copy = {name: place.copy() for name, place in self.places.items()}
        items_copy = {name: item.copy() for name, item in self.items.items()}
        return WorldState(places_copy, items_copy)

    def __eq__(self, other):
        return self.places == other.places and self.items == other.items

    def __hash__(self):
        places_items = tuple(sorted((name, self.places[name]) for name in sorted(self.places)))
        items_items = tuple(sorted((name, self.items[name]) for name in sorted(self.items)))
        return hash((places_items, items_items))

def plan(goals, npc_state, world_state, actions):
    open_list = []
    closed_list = set()
    came_from = {}

    counter = itertools.count()

    # Function to calculate heuristic
    def heuristic(npc_state, world_state, achieved_goals):
        # Sum the weights of unachieved goals
        return sum(goal.weight for goal in goals if goal.name not in achieved_goals and goal.weight > 0)

    # 초기 상태에 대한 목표 달성 상태 설정
    initial_achieved_goals = frozenset()
    initial_heuristic = heuristic(npc_state, world_state, initial_achieved_goals)
    heappush(open_list, (initial_heuristic, 0, next(counter), npc_state, world_state, initial_achieved_goals))
    came_from[(npc_state, world_state, initial_achieved_goals)] = None
    cost_so_far = {(npc_state, world_state, initial_achieved_goals): 0}

    while open_list:
        _, current_cost, _, current_npc_state, current_world_state, achieved_goals = heappop(open_list)
        state_key = (current_npc_state, current_world_state, achieved_goals)

        if state_key in closed_list:
            continue

        closed_list.add(state_key)

        # Check if all goals are met
        if all(goal.condition(current_npc_state, current_world_state) for goal in goals if goal.weight > 0):
            return reconstruct_plan(came_from, state_key)

        for action in actions:
            if action.is_applicable(current_npc_state, current_world_state):
                new_npc_state, new_world_state = action.apply(current_npc_state, current_world_state)

                action_cost = sum(abs(cost) for cost in action.cost.values())
                new_cost = cost_so_far[state_key] + action_cost

                # Update achieved goals
                new_achieved_goals = set(achieved_goals)
                for goal in goals:
                    if goal.name not in new_achieved_goals and goal.condition(new_npc_state, new_world_state):
                        new_achieved_goals.add(goal.name)
                new_achieved_goals = frozenset(new_achieved_goals)

                new_state_key = (new_npc_state, new_world_state, new_achieved_goals)

                if new_state_key not in cost_so_far or new_cost < cost_so_far[new_state_key]:
                    cost_so_far[new_state_key] = new_cost
                    h = heuristic(new_npc_state, new_world_state, new_achieved_goals)
                    priority = new_cost + h
                    heappush(open_list, (priority, new_cost, next(counter), new_npc_state, new_world_state, new_achieved_goals))
                    came_from[new_state_key] = (state_key, action)

    return None

# 플랜 재구성 함수 수정
def reconstruct_plan(came_from, current_state):
    actions = []
    while came_from[current_state] is not None:
        prev_state, action = came_from[current_state]
        actions.append(action)
        current_state = prev_state
    actions.reverse()
    return actions

문장 파서

In [9]:
def parse_sentence_to_goal(sentence, actions, weight=1):

    # 액션 딕셔너리 생성 (이름을 키로)
    action_dict = {action.name: action for action in actions}

    # 문장이 None이거나 "none"일 경우 에러 메시지 없이 None 반환
    if sentence is None or sentence.strip().lower() == "none":
        return None

    # 문장 끝의 마침표 제거
    sentence = sentence.strip().rstrip('.')

    # "Do <action> in/on <location>" 형태의 문장 처리 (in 또는 on 사용 가능)
    match_action_at_location = re.match(r'Do\s+(.+?)\s+(in|on)\s+(.+)', sentence)
    if match_action_at_location:
        action_name = match_action_at_location.group(1).strip()
        location = match_action_at_location.group(3).strip()
        goal_name = f"Do_{action_name}_at_{location}"

        # 액션 딕셔너리에서 액션 찾기
        if action_name in action_dict:
            action = action_dict[action_name]

            # 조건 함수 정의
            def condition(npc_state, world_state):
                if npc_state.lower_body.get("location") != location:
                    return False
                # 액션의 효과를 기반으로 상태를 확인
                for k, v in action.effects.items():
                    if not check_effect_applied(k, v, npc_state, world_state):
                        return False
                return True

            return Goal(name=goal_name, condition=condition, weight=weight)
        else:
            print(f"액션 '{action_name}'을 찾을 수 없습니다.")
            return None

    # "Change <state> of <object> to <value>" 형태의 문장 처리
    match_change_state = re.match(r'Change\s+(.+?)\s+of\s+(.+?)\s+to\s+(.+)', sentence)
    if match_change_state:
        state_key = match_change_state.group(1).strip()
        obj_name = match_change_state.group(2).strip()
        desired_value = match_change_state.group(3).strip()
        goal_name = f"Change_{state_key}_of_{obj_name}_to_{desired_value}"

        # 조건 함수 정의
        def condition(npc_state, world_state):
            return check_state(obj_name, state_key, desired_value, npc_state, world_state)

        return Goal(name=goal_name, condition=condition, weight=weight)

    # "Go to <location>" 형태의 문장 처리
    match_go_to_location = re.match(r'Go\s+to\s+(.+)', sentence)
    if match_go_to_location:
        location = match_go_to_location.group(1).strip()
        goal_name = f"Go_to_{location}"

        def condition(npc_state, world_state):
            return npc_state.lower_body.get("location") == location

        return Goal(name=goal_name, condition=condition, weight=weight)

    # "Pick up <item>" 형태의 문장 처리
    match_pick_up = re.match(r'Pick\s+up\s+(.+)', sentence)
    if match_pick_up:
        item_name = match_pick_up.group(1).strip()
        goal_name = f"Pick_up_{item_name}"

        def condition(npc_state, world_state):
            return item_name in npc_state.inventory

        return Goal(name=goal_name, condition=condition, weight=weight)

    # "Drop <item> at <location>" 형태의 문장 처리
    match_drop_item = re.match(r'Drop\s+(.+?)\s+at\s+(.+)', sentence)
    if match_drop_item:
        item_name = match_drop_item.group(1).strip()
        location = match_drop_item.group(2).strip()
        goal_name = f"Drop_{item_name}_at_{location}"

        def condition(npc_state, world_state):
            place = world_state.places.get(location)
            return (
                npc_state.lower_body.get("location") == location and
                item_name not in npc_state.inventory and
                place and item_name in place.inventory
            )

        return Goal(name=goal_name, condition=condition, weight=weight)

    # "Do <action>" 형태의 문장 처리
    match_action = re.match(r'Do\s+(.+)', sentence)
    if match_action:
        action_name = match_action.group(1).strip()
        goal_name = f"Do_{action_name}"

        # 액션 딕셔너리에서 액션 찾기
        if action_name in action_dict:
            action = action_dict[action_name]

            # 조건 함수 정의: 액션의 효과가 적용되었는지 확인
            def condition(npc_state, world_state):
                if not action.effects:
                    return npc_state.state_data.get(f"did_{action_name}", False)
                for k, v in action.effects.items():
                    if not check_effect_applied(k, v, npc_state, world_state):
                        return False
                return True

            return Goal(name=goal_name, condition=condition, weight=weight)
        else:
            print(f"액션 '{action_name}'을 찾을 수 없습니다.")
            return None

    # 파싱 실패 시 에러 메시지 출력
    print("문장 파싱 에러")
    return None

# 효과가 적용되었는지 확인하는 헬퍼 함수
def check_effect_applied(effect_key, effect_value, npc_state, world_state):
    if effect_key in npc_state.upper_body:
        return npc_state.upper_body.get(effect_key) == effect_value
    elif effect_key in npc_state.lower_body:
        return npc_state.lower_body.get(effect_key) == effect_value
    elif effect_key in npc_state.resources:
        return npc_state.resources.get(effect_key) == effect_value
    elif effect_key == "hold":
        return npc_state.upper_body.get("hold") == effect_value
    elif effect_key.startswith("place_state:"):
        _, place_name, state_key = effect_key.split(":")
        place = world_state.places.get(place_name)
        return place and place.state.get(state_key) == effect_value
    elif effect_key.startswith("item_state:"):
        _, item_name, state_key = effect_key.split(":")
        item = world_state.items.get(item_name)
        return item and item.state.get(state_key) == effect_value
    elif effect_key == "pose":
        return npc_state.lower_body.get("pose") == effect_value
    elif effect_key == "use_item":
        # 아이템이 인벤토리에 없는지 확인 (사용되었으므로)
        item_name = effect_value
        return item_name not in npc_state.inventory
    elif effect_key == "pickup_item":
        # 아이템이 인벤토리에 있는지 확인
        item_name = effect_value
        return item_name in npc_state.inventory
    elif effect_key == "drop_item":
        # 아이템이 현재 장소의 인벤토리에 있는지 확인
        item_name = effect_value
        current_location = npc_state.lower_body.get("location")
        place = world_state.places.get(current_location)
        return place and item_name in place.inventory
    else:
        # 기타 상태 데이터 확인
        return npc_state.state_data.get(effect_key) == effect_value

# 상태를 확인하는 헬퍼 함수
def check_state(obj_name, state_key, desired_value, npc_state, world_state):
    if obj_name == 'NPC':
        if state_key in npc_state.upper_body:
            return npc_state.upper_body.get(state_key) == desired_value
        elif state_key in npc_state.lower_body:
            return npc_state.lower_body.get(state_key) == desired_value
        else:
            return npc_state.state_data.get(state_key) == desired_value
    elif obj_name in world_state.places:
        place = world_state.places[obj_name]
        return place.state.get(state_key) == desired_value
    elif obj_name in world_state.items:
        item = world_state.items[obj_name]
        return item.state.get(state_key) == desired_value
    else:
        return False


Action 생성기

In [3]:
def create_pick_action(item_name):
    return Action(
        name=f"pick_{item_name}",
        conditions={
            "hold": "none",
            "pickup_item": lambda npc_state, world_state: item_name in world_state.places[npc_state.lower_body["location"]].inventory
        },
        effects={
            "hold": item_name,
            "pickup_item": item_name
        },
        cost={"time": 0.1, "health": 0, "mental": 0}
    )
def create_drop_action(item_name):
    return Action(
        name=f"drop_{item_name}",
        conditions={
            "hold": item_name
        },
        effects={
            "hold": "none",
            "drop_item": item_name
        },
        cost={"time": 0.1, "health": 0, "mental": 0}
    )
def create_move_action(from_place, to_place, time_cost, health_cost):
    return Action(
        name=f"{from_place}_to_{to_place}",
        conditions={
            "pose": "stand",
            "location": from_place
        },
        effects={
            "location": to_place
        },
        cost={"time": time_cost, "health": health_cost, "mental": 0}
    )

GOAP 테스트

In [14]:
# 장소 및 아이템 정의
table = Place("table", inventory=["map", "snack"], state={})
sofa = Place("sofa", inventory=["pillow"], state={})
door = Place("door", state={"door_state": "closed"})

# 장소 간 연결 정보 정의
place_connections = {
    "sofa": ["table", "door"],
    "table": ["sofa", "door"],
    "door": ["sofa", "table"]
}

places = {"table": table, "sofa": sofa, "door": door}

snack = Item(
    "snack",
    behaviors={"use": {"conditions": {}, "effects": {"health": 10}}}
)

map = Item(
    "map",
    behaviors={}
)

pillow = Item(
    "pillow",
    behaviors={}
)

items = {"snack": snack, "map": map, "pillow": pillow}

# 초기 상태 정의
npc_state = NPCState(
    upper_body={"hold": "none"},
    lower_body={"location": "sofa", "pose": "stand"},
    resources={"time": 0, "health": 100, "mental": 100},
    inventory=[]
)

world_state = WorldState(places, items)

# 액션 정의
actions = [
    Action(
        name="eat_snack",
        conditions={
            "hold": "snack"
        },
        effects={
            "use_item": "snack",
            # "did_eat_snack": True  # 상태 데이터 변경 대신 자원 변화로 효과 반영
        },
        cost={"time": 0.5, "health": 0, "mental": 0}
    ),
    Action(
        name="open_door",
        conditions={
            "location": "door",
            "door_closed": lambda npc_state, world_state: world_state.places["door"].state.get("door_state") == "closed"
        },
        effects={"place_state:door:door_state": "open"},
        cost={"time": 0.5, "health": 0, "mental": 0}
    ),
    Action(
        name="greet",
        conditions={
            "hold": "none",
            "pose": "stand"
        },
        effects={
            "did_greet": True  # 액션이 수행되었음을 나타내는 효과 추가
        },
        cost={"time": 1, "health": 1, "mental": 1}
    ),
    Action(
        name="sit_sofa",
        conditions={
            "location": "sofa",
            "pose": "stand"
        },
        effects={
            "pose": "sit"
        },
        cost={"time": 1, "health": 1, "mental": 1}
    ),
    Action(
        name="stand_sofa",
        conditions={
            "location": "sofa",
            "pose": "sit"
        },
        effects={
            "pose": "stand"
        },
        cost={"time": 1, "health": 3, "mental": 3}
    ),
]

# 아이템마다 픽업 액션 생성
for item_name in items.keys():
    actions.append(create_pick_action(item_name))

# 모든 아이템에 대해 드롭 액션 생성
for item_name in items.keys():
    actions.append(create_drop_action(item_name))

# 연결된 장소 간 이동 액션 생성
for from_place, to_places in place_connections.items():
    for to_place in to_places:
        # 장소에 따라 비용 정의 (예시로 간단히 설정)
        if (from_place, to_place) in [("sofa", "door"), ("door", "sofa")]:
            time_cost = 0.5
            health_cost = 0.5
        elif (from_place, to_place) in [("sofa", "table"), ("table", "sofa")]:
            time_cost = 0.5
            health_cost = 0.5
        else:
            time_cost = 1.5
            health_cost = 1.5

        actions.append(create_move_action(from_place, to_place, time_cost, health_cost))

def goap(goal_sentences_with_weights, npc_state, world_state):

    goals = []

    for sentence, weight in goal_sentences_with_weights:
        goal = parse_sentence_to_goal(sentence, actions, weight=weight)
        if goal:
            goals.append(goal)

    # 플래닝 실행
    plan_result = plan(goals, npc_state, world_state, actions)

    current_npc_state = npc_state.copy()
    current_world_state = world_state.copy()

    # 결과 출력 및 플랜 실행
    if plan_result:
        print("플랜이 성공적으로 생성되었습니다:")
        total_cost = {"time": 0, "health": 0, "mental": 0}
        for action in plan_result:
            print(f"- {action.name}")
            for k in total_cost.keys():
                total_cost[k] += action.cost.get(k, 0)
            current_npc_state, current_world_state = action.apply(current_npc_state, current_world_state)

        print("\n총 비용:")
        for k, v in total_cost.items():
            print(f"{k}: {v}")
    else:
        print("목표를 달성할 수 없습니다. 가만히 있습니다.")
        plan_result = []
    return plan_result, current_npc_state, current_world_state

# 목표 정의 (가중치 포함)
goal_example = [
    ("Do sit_sofa", 10),
    #("Go to table", 1)
    #("Do open_door", 1)
    ("Do greet", 1)
]

goap(goal_example, npc_state, world_state)


플랜이 성공적으로 생성되었습니다:
- greet
- sit_sofa

총 비용:
time: 2
health: 2
mental: 2


([<__main__.Action at 0x7d490f015c30>, <__main__.Action at 0x7d490f015b10>],
 <__main__.WorldState at 0x7d49058133a0>)

## Fast GOAP (Unity)

In [25]:
from collections import deque
from heapq import heappush, heappop
import itertools
import re

# 헬퍼 함수: 중첩된 dict와 list를 해시 가능한 불변 타입으로 변환
def make_hashable(obj):
    if isinstance(obj, dict):
        return tuple(sorted((k, make_hashable(v)) for k, v in obj.items()))
    elif isinstance(obj, list):
        return tuple(make_hashable(x) for x in obj)
    elif isinstance(obj, set):
        return frozenset(make_hashable(x) for x in obj)
    else:
        return obj

# 장소 클래스 정의 (상태 저장 기능 추가)
class Place:
    def __init__(self, name, inventory=None, state=None):
        self.name = name
        self.inventory = inventory if inventory is not None else []
        self.state = state if state is not None else {}

    def copy(self):
        return Place(self.name, self.inventory.copy(), self.state.copy())

    def __eq__(self, other):
        return self.name == other.name and self.inventory == other.inventory and self.state == other.state

    def __hash__(self):
        return hash((self.name, tuple(sorted(self.inventory)), make_hashable(self.state)))

    def __repr__(self):
        return f"Place(name={self.name}, inventory={self.inventory}, state={self.state})"

# 아이템 클래스 정의 (상태 저장 기능 추가)
class Item:
    def __init__(self, name, behaviors, state=None):
        self.name = name
        self.behaviors = behaviors  # {'behavior_name': {'conditions': {...}, 'effects': {...}}}
        self.state = state if state is not None else {}

    def copy(self):
        return Item(self.name, self.behaviors.copy(), self.state.copy())

    def __eq__(self, other):
        return self.name == other.name and self.behaviors == other.behaviors and self.state == other.state

    def __hash__(self):
        return hash((self.name, make_hashable(self.behaviors), make_hashable(self.state)))

    def __repr__(self):
        return f"Item(name={self.name}, behaviors={self.behaviors}, state={self.state})"

# Goal 클래스 정의 (가중치 추가)
class Goal:
    def __init__(self, name, condition, weight=1, required_state=None):
        self.name = name
        self.condition = condition  # 함수: npc_state와 world_state를 받아서 bool 반환
        self.achieved = False
        self.weight = weight  # 가중치 (높을수록 우선순위 높음)
        self.required_state = required_state if required_state else {}

# Action 클래스 정의
class Action:
    def __init__(self, name, conditions, effects, cost):
        self.name = name
        self.conditions = conditions  # 조건: 상태 키와 값 또는 함수
        self.effects = effects        # 효과: 상태 키와 값 또는 함수
        self.cost = cost

    def is_applicable(self, npc_state, world_state):
        conditions_met = True
        for k, v in self.conditions.items():
            if callable(v):
                if not v(npc_state, world_state):
                    conditions_met = False
                    break
            else:
                if k in npc_state.upper_body:
                    if npc_state.upper_body.get(k) != v:
                        conditions_met = False
                        break
                elif k in npc_state.lower_body:
                    if npc_state.lower_body.get(k) != v:
                        conditions_met = False
                        break
                elif k in npc_state.resources:
                    if npc_state.resources.get(k) != v:
                        conditions_met = False
                        break
                elif k in npc_state.state_data:
                    if npc_state.state_data.get(k) != v:
                        conditions_met = False
                        break
                else:
                    conditions_met = False
                    break
        if not conditions_met:
            return False
        for resource, cost_value in self.cost.items():
            if resource == "time":
                continue  # Time은 누적되므로 체크하지 않습니다.
            current_value = npc_state.resources.get(resource)
            if current_value is not None and (current_value - cost_value) < 0:
                return False
        return True

    def apply(self, npc_state, world_state):
        new_npc_state = npc_state.copy()
        new_world_state = world_state.copy()

        # 액션의 효과 적용
        for k, v in self.effects.items():
            if k in new_npc_state.upper_body:
                new_npc_state.upper_body[k] = v
            elif k in new_npc_state.lower_body:
                new_npc_state.lower_body[k] = v
            elif k in new_npc_state.resources:
                new_npc_state.resources[k] = v
            elif k == "places":
                new_world_state.places = v
            elif k == "used_weapon":
                new_npc_state.state_data['used_weapon'] = v
            elif k == "pickup_item":
                # 아이템 획득 처리
                item_name = v
                current_location = new_npc_state.lower_body["location"]
                if item_name in new_world_state.places[current_location].inventory:
                    new_world_state.places[current_location].inventory.remove(item_name)
                    new_npc_state.inventory.append(item_name)
            elif k == "use_item":
                # 아이템 사용 처리
                item_name = v
                apply_use_item(new_npc_state, new_world_state, item_name)
                print(f"아이템 '{item_name}'을(를) 사용했습니다. 상태 데이터: {new_npc_state.state_data}")
            elif k.startswith("item_state:"):
                # 아이템 상태 변경
                _, item_name, state_key = k.split(":")
                if item_name in new_world_state.items:
                    item = new_world_state.items[item_name]
                    item.state[state_key] = v
            elif k.startswith("place_state:"):
                # 장소 상태 변경
                _, place_name, state_key = k.split(":")
                if place_name in new_world_state.places:
                    place = new_world_state.places[place_name]
                    place.state[state_key] = v
            elif k == "drop_item":
                # 아이템을 현재 장소의 인벤토리에 추가
                item_name = v
                if item_name in new_npc_state.inventory:
                    current_location = new_npc_state.lower_body.get("location")
                    new_npc_state.inventory.remove(item_name)
                    new_world_state.places[current_location].inventory.append(item_name)
            else:
                new_npc_state.state_data[k] = v

        # 자원 비용 적용
        for resource, cost_value in self.cost.items():
            if resource in new_npc_state.resources:
                if resource == "time":
                    new_npc_state.resources[resource] += cost_value  # Time increases
                else:
                    new_npc_state.resources[resource] -= cost_value  # Resources decrease

        print(f"액션 '{self.name}' 적용 후 NPC 상태: {new_npc_state}")
        print(f"액션 '{self.name}' 적용 후 월드 상태: {new_world_state.places}")
        return new_npc_state, new_world_state

    def __repr__(self):
        return f"Action(name={self.name}, conditions={self.conditions}, effects={self.effects}, cost={self.cost})"

# NPC 상태 정의
class NPCState:
    def __init__(self, upper_body, lower_body, resources, inventory=None, state_data=None):
        self.upper_body = upper_body
        self.lower_body = lower_body
        self.resources = resources
        self.inventory = inventory if inventory is not None else []
        self.state_data = state_data if state_data is not None else {}

    def get(self, key):
        if key in self.upper_body:
            return self.upper_body[key]
        elif key in self.lower_body:
            return self.lower_body[key]
        elif key in self.resources:
            return self.resources[key]
        elif key in self.state_data:
            return self.state_data[key]
        return None

    def copy(self):
        # Round resource values to prevent floating-point errors
        rounded_resources = {k: round(v, 2) for k, v in self.resources.items()}
        return NPCState(
            self.upper_body.copy(),
            self.lower_body.copy(),
            rounded_resources,
            self.inventory.copy(),
            self.state_data.copy()
        )

    def __eq__(self, other):
        # Exclude 'time', 'health', and 'mental' from comparison
        self_resources = {k: v for k, v in self.resources.items() if k not in ['time', 'health', 'mental']}
        other_resources = {k: v for k, v in other.resources.items() if k not in ['time', 'health', 'mental']}
        return (
            self.upper_body == other.upper_body and
            self.lower_body == other.lower_body and
            self_resources == other_resources and
            self.inventory == other.inventory and
            self.state_data == other.state_data
        )

    def __hash__(self):
        # Exclude 'time', 'health', and 'mental' from hashing
        resources_without_time = make_hashable({k: v for k, v in self.resources.items() if k not in ['time', 'health', 'mental']})
        return hash((
            make_hashable(self.upper_body),
            make_hashable(self.lower_body),
            resources_without_time,
            tuple(sorted(self.inventory)),
            make_hashable(self.state_data)
        ))

    def __repr__(self):
        return (f"NPCState(upper_body={self.upper_body}, lower_body={self.lower_body}, "
                f"resources={self.resources}, inventory={self.inventory}, state_data={self.state_data})")

# World 상태 정의
class WorldState:
    def __init__(self, places, items):
        self.places = places
        self.items = items

    def copy(self):
        places_copy = {name: place.copy() for name, place in self.places.items()}
        items_copy = {name: item.copy() for name, item in self.items.items()}
        return WorldState(places_copy, items_copy)

    def __eq__(self, other):
        return self.places == other.places and self.items == other.items

    def __hash__(self):
        return hash((
            tuple(sorted((name, make_hashable(place)) for name, place in self.places.items())),
            tuple(sorted((name, make_hashable(item)) for name, item in self.items.items()))
        ))

    def __repr__(self):
        return f"WorldState(places={self.places}, items={self.items})"

# 효과가 적용되었는지 확인하는 헬퍼 함수
def check_effect_applied(effect_key, effect_value, npc_state, world_state):
    if effect_key in npc_state.upper_body:
        result = npc_state.upper_body.get(effect_key) == effect_value
    elif effect_key in npc_state.lower_body:
        result = npc_state.lower_body.get(effect_key) == effect_value
    elif effect_key in npc_state.resources:
        result = npc_state.resources.get(effect_key) == effect_value
    elif effect_key == "hold":
        result = npc_state.upper_body.get("hold") == effect_value
    elif effect_key.startswith("place_state:"):
        _, place_name, state_key = effect_key.split(":")
        place = world_state.places.get(place_name)
        result = place and place.state.get(state_key) == effect_value
    elif effect_key.startswith("item_state:"):
        _, item_name, state_key = effect_key.split(":")
        item = world_state.items.get(item_name)
        result = item and item.state.get(state_key) == effect_value
    elif effect_key == "pose":
        result = npc_state.lower_body.get("pose") == effect_value
    elif effect_key == "use_item":
        # 'use_item' 효과를 확인하기 위해 'used_snack' 플래그 확인
        result = npc_state.state_data.get(f'used_{effect_value}', False)
    elif effect_key == "pickup_item":
        # 아이템이 인벤토리에 있는지 확인
        item_name = effect_value
        result = item_name in npc_state.inventory
    elif effect_key == "drop_item":
        # 아이템이 현재 장소의 인벤토리에 있는지 확인
        item_name = effect_value
        current_location = npc_state.lower_body.get("location")
        place = world_state.places.get(current_location)
        result = place and item_name in place.inventory
    else:
        # 기타 상태 데이터 확인
        result = npc_state.state_data.get(effect_key) == effect_value

    print(f"효과 '{effect_key}': '{effect_value}' 확인 결과: {result}")
    return result

# 상태를 확인하는 헬퍼 함수
def check_state(obj_name, state_key, desired_value, npc_state, world_state):
    if obj_name == 'NPC':
        if state_key in npc_state.upper_body:
            return npc_state.upper_body.get(state_key) == desired_value
        elif state_key in npc_state.lower_body:
            return npc_state.lower_body.get(state_key) == desired_value
        else:
            return npc_state.state_data.get(state_key) == desired_value
    elif obj_name in world_state.places:
        place = world_state.places[obj_name]
        return place.state.get(state_key) == desired_value
    elif obj_name in world_state.items:
        item = world_state.items[obj_name]
        return item.state.get(state_key) == desired_value
    else:
        return False

# 문장 파싱 및 Goal 생성 함수 (액션 정보 활용)
def parse_sentence_to_goal(sentence, actions, weight=1):

    # 액션 딕셔너리 생성 (이름을 키로)
    action_dict = {action.name: action for action in actions}

    # 문장이 None이거나 "none"일 경우 에러 메시지 없이 None 반환
    if sentence is None or sentence.strip().lower() == "none":
        return None

    # 문장 끝의 마침표 제거
    sentence = sentence.strip().rstrip('.')

    # "Do <action> in/on <location>" 형태의 문장 처리 (in 또는 on 사용 가능)
    match_action_at_location = re.match(r'Do\s+(.+?)\s+(in|on)\s+(.+)', sentence)
    if match_action_at_location:
        action_name = match_action_at_location.group(1).strip()
        location = match_action_at_location.group(3).strip()
        goal_name = f"Do_{action_name}_at_{location}"

        # 액션 딕셔너리에서 액션 찾기
        if action_name in action_dict:
            action = action_dict[action_name]

            # 조건 함수 정의
            def condition(npc_state, world_state):
                if npc_state.lower_body.get("location") != location:
                    return False
                # 액션의 효과를 기반으로 상태를 확인
                for k, v in action.effects.items():
                    if not check_effect_applied(k, v, npc_state, world_state):
                        return False
                return True

            return Goal(name=goal_name, condition=condition, weight=weight, required_state=action.effects)
        else:
            print(f"액션 '{action_name}'을 찾을 수 없습니다.")
            return None

    # "Change <state> of <object> to <value>" 형태의 문장 처리
    match_change_state = re.match(r'Change\s+(.+?)\s+of\s+(.+?)\s+to\s+(.+)', sentence)
    if match_change_state:
        state_key = match_change_state.group(1).strip()
        obj_name = match_change_state.group(2).strip()
        desired_value = match_change_state.group(3).strip()
        goal_name = f"Change_{state_key}_of_{obj_name}_to_{desired_value}"

        # 조건 함수 정의
        def condition(npc_state, world_state):
            return check_state(obj_name, state_key, desired_value, npc_state, world_state)

        return Goal(name=goal_name, condition=condition, weight=weight)

    # "Go to <location>" 형태의 문장 처리
    match_go_to_location = re.match(r'Go\s+to\s+(.+)', sentence)
    if match_go_to_location:
        location = match_go_to_location.group(1).strip()
        goal_name = f"Go_to_{location}"

        def condition(npc_state, world_state):
            return npc_state.lower_body.get("location") == location

        return Goal(name=goal_name, condition=condition, weight=weight)

    # "Pick up <item>" 형태의 문장 처리
    match_pick_up = re.match(r'Pick\s+up\s+(.+)', sentence)
    if match_pick_up:
        item_name = match_pick_up.group(1).strip()
        goal_name = f"Pick_up_{item_name}"

        def condition(npc_state, world_state):
            return item_name in npc_state.inventory

        return Goal(name=goal_name, condition=condition, weight=weight)

    # "Drop <item> at <location>" 형태의 문장 처리
    match_drop_item = re.match(r'Drop\s+(.+?)\s+at\s+(.+)', sentence)
    if match_drop_item:
        item_name = match_drop_item.group(1).strip()
        location = match_drop_item.group(2).strip()
        goal_name = f"Drop_{item_name}_at_{location}"

        def condition(npc_state, world_state):
            place = world_state.places.get(location)
            return (
                npc_state.lower_body.get("location") == location and
                item_name not in npc_state.inventory and
                place and item_name in place.inventory
            )

        return Goal(name=goal_name, condition=condition, weight=weight)

    # "Do <action>" 형태의 문장 처리
    match_action = re.match(r'Do\s+(.+)', sentence)
    if match_action:
        action_name = match_action.group(1).strip()
        goal_name = f"Do_{action_name}"

        # 액션 딕셔너리에서 액션 찾기
        if action_name in action_dict:
            action = action_dict[action_name]

            # 조건 함수 정의: 액션의 효과가 적용되었는지 확인
            def condition(npc_state, world_state):
                if not action.effects:
                    return npc_state.state_data.get(f"did_{action_name}", False)
                for k, v in action.effects.items():
                    if not check_effect_applied(k, v, npc_state, world_state):
                        return False
                return True

            return Goal(name=goal_name, condition=condition, weight=weight, required_state=action.effects)
        else:
            print(f"액션 '{action_name}'을 찾을 수 없습니다.")
            return None

    # 파싱 실패 시 에러 메시지 출력
    print("문장 파싱 에러")
    return None

# 액션 생성 함수
def create_pick_action(item_name):
    return Action(
        name=f"pick_{item_name}",
        conditions={
            "hold": "none",
            "pickup_item": lambda npc_state, world_state: item_name in world_state.places[npc_state.lower_body['location']].inventory
        },
        effects={
            "hold": item_name,
            "pickup_item": item_name
        },
        cost={"time": 0.1, "health": 0, "mental": 0}
    )

def create_drop_action(item_name):
    return Action(
        name=f"drop_{item_name}",
        conditions={
            "hold": item_name
        },
        effects={
            "hold": "none",
            "drop_item": item_name
        },
        cost={"time": 0.1, "health": 0, "mental": 0}
    )

def create_move_action(from_place, to_place, time_cost, health_cost):
    return Action(
        name=f"move_{from_place}_to_{to_place}",
        conditions={
            "pose": "stand",
            "location": from_place
        },
        effects={
            "location": to_place
        },
        cost={"time": time_cost, "health": health_cost, "mental": 0}
    )

# 아이템 사용 처리 함수
def apply_use_item(new_npc_state, new_world_state, item_name):
    if item_name in new_npc_state.inventory:
        # 아이템 사용 효과를 상태에 반영
        new_npc_state.inventory.remove(item_name)  # 아이템 사용 후 인벤토리에서 제거
        item = new_world_state.items[item_name]
        behavior = item.behaviors.get("use", {})
        effects = behavior.get("effects", {})
        for effect_k, effect_v in effects.items():
            if effect_k in new_npc_state.resources:
                new_npc_state.resources[effect_k] += effect_v
            else:
                new_npc_state.state_data[effect_k] = effect_v
        # 'use_item' 효과를 나타내기 위해 상태 데이터 추가
        new_npc_state.state_data[f'used_{item_name}'] = True
        print(f"아이템 '{item_name}' 사용 후 상태 데이터: {new_npc_state.state_data}")


# 플래너 함수
def plan(goals, npc_state, world_state, actions):
    open_list = []
    closed_list = set()
    came_from = {}

    counter = itertools.count()

    # Function to calculate heuristic
    def heuristic(npc_state, world_state, achieved_goals):
        # Sum the weights of unachieved goals
        return sum(goal.weight for goal in goals if goal.name not in achieved_goals and goal.weight > 0)

    # 초기 상태에 대한 목표 달성 상태 설정
    initial_achieved_goals = frozenset()
    initial_heuristic = heuristic(npc_state, world_state, initial_achieved_goals)
    heappush(open_list, (initial_heuristic, 0, next(counter), npc_state, world_state, initial_achieved_goals))
    came_from[(make_hashable(npc_state.upper_body), make_hashable(npc_state.lower_body),
               make_hashable({k: v for k, v in npc_state.resources.items() if k not in ['time', 'health', 'mental']}),
               tuple(sorted(npc_state.inventory)), make_hashable(npc_state.state_data),
               make_hashable(world_state.places), make_hashable(world_state.items),
               initial_achieved_goals)] = None
    cost_so_far = {(make_hashable(npc_state.upper_body), make_hashable(npc_state.lower_body),
                   make_hashable({k: v for k, v in npc_state.resources.items() if k not in ['time', 'health', 'mental']}),
                   tuple(sorted(npc_state.inventory)), make_hashable(npc_state.state_data),
                   make_hashable(world_state.places), make_hashable(world_state.items),
                   initial_achieved_goals): 0}

    while open_list:
        _, current_cost, _, current_npc_state, current_world_state, achieved_goals = heappop(open_list)
        state_key = (
            make_hashable(current_npc_state.upper_body),
            make_hashable(current_npc_state.lower_body),
            make_hashable({k: v for k, v in current_npc_state.resources.items() if k not in ['time', 'health', 'mental']}),
            tuple(sorted(current_npc_state.inventory)),
            make_hashable(current_npc_state.state_data),
            make_hashable(current_world_state.places),
            make_hashable(current_world_state.items),
            achieved_goals
        )

        if state_key in closed_list:
            continue

        closed_list.add(state_key)

        # 디버깅: 현재 상태와 달성된 목표 출력
        print(f"\n현재 상태: {current_npc_state}")
        print(f"달성된 목표: {achieved_goals}")

        # Check if all goals are met
        all_goals_met = all(goal.condition(current_npc_state, current_world_state) for goal in goals if goal.weight > 0)
        print(f"모든 목표 달성 여부: {all_goals_met}")
        if all_goals_met:
            return reconstruct_plan(came_from, state_key)

        for action in actions:
            if action.is_applicable(current_npc_state, current_world_state):
                new_npc_state, new_world_state = action.apply(current_npc_state, current_world_state)

                action_cost = sum(abs(cost) for cost in action.cost.values())
                new_cost = current_cost + action_cost

                # Update achieved goals
                new_achieved_goals = set(achieved_goals)
                for goal in goals:
                    if goal.name not in new_achieved_goals and goal.condition(new_npc_state, new_world_state):
                        new_achieved_goals.add(goal.name)
                        print(f"목표 '{goal.name}'이(가) 달성되었습니다.")

                new_achieved_goals = frozenset(new_achieved_goals)

                new_state_key = (
                    make_hashable(new_npc_state.upper_body),
                    make_hashable(new_npc_state.lower_body),
                    make_hashable({k: v for k, v in new_npc_state.resources.items() if k not in ['time', 'health', 'mental']}),
                    tuple(sorted(new_npc_state.inventory)),
                    make_hashable(new_npc_state.state_data),
                    make_hashable(new_world_state.places),
                    make_hashable(new_world_state.items),
                    new_achieved_goals
                )

                if new_state_key not in cost_so_far or new_cost < cost_so_far[new_state_key]:
                    cost_so_far[new_state_key] = new_cost
                    h = heuristic(new_npc_state, new_world_state, new_achieved_goals)
                    priority = new_cost + h
                    heappush(open_list, (priority, new_cost, next(counter), new_npc_state, new_world_state, new_achieved_goals))
                    came_from[new_state_key] = (state_key, action)

    return None

# 플랜 재구성 함수
def reconstruct_plan(came_from, current_state):
    actions = []
    while came_from[current_state] is not None:
        prev_state, action = came_from[current_state]
        actions.append(action)
        current_state = prev_state
    actions.reverse()
    return actions

In [26]:
# 장소 및 아이템 정의
table = Place("table", inventory=["map", "snack"], state={})
sofa = Place("sofa", inventory=["pillow"], state={})
door = Place("door", state={"door_state": "closed"})

# 장소 간 연결 정보 정의
place_connections = {
    "sofa": ["table", "door"],
    "table": ["sofa", "door"],
    "door": ["sofa", "table"]
}

places = {"table": table, "sofa": sofa, "door": door}

snack = Item(
    "snack",
    behaviors={"use": {"conditions": {}, "effects": {"health": 10}}}
)

map_item = Item(
    "map",
    behaviors={}
)

pillow = Item(
    "pillow",
    behaviors={}
)

items = {"snack": snack, "map": map_item, "pillow": pillow}

# 초기 상태 정의
npc_state = NPCState(
    upper_body={"hold": "none"},
    lower_body={"location": "sofa", "pose": "stand"},
    resources={"time": 0, "health": 100, "mental": 100},
    inventory=[]
)

world_state = WorldState(places, items)

# 액션 정의
actions = [
    Action(
        name="eat_snack",
        conditions={
            "hold": "snack"
        },
        effects={
            "use_item": "snack",
            "hold": "none",
            "used_snack": True  # 사용된 스낵을 명시적으로 표시
        },
        cost={"time": 0.5, "health": 0, "mental": 0}
    ),
    Action(
        name="open_door",
        conditions={
            "location": "door",
            "door_closed": lambda npc_state, world_state: world_state.places["door"].state.get("door_state") == "closed"
        },
        effects={"place_state:door:door_state": "open"},
        cost={"time": 0.5, "health": 0, "mental": 0}
    ),
    Action(
        name="greet",
        conditions={
            "hold": "none",
            "pose": "stand"
        },
        effects={
            "did_greet": True  # 액션이 수행되었음을 나타내는 효과 추가
        },
        cost={"time": 1, "health": 1, "mental": 1}
    ),
    Action(
        name="sit_sofa",
        conditions={
            "location": "sofa",
            "pose": "stand"
        },
        effects={
            "pose": "sit"
        },
        cost={"time": 1, "health": 1, "mental": 1}
    ),
    Action(
        name="stand_sofa",
        conditions={
            "location": "sofa",
            "pose": "sit"
        },
        effects={
            "pose": "stand"
        },
        cost={"time": 1, "health": 3, "mental": 3}
    ),
]

# 아이템마다 픽업 액션 생성
for item_name in items.keys():
    actions.append(create_pick_action(item_name))

# 모든 아이템에 대해 드롭 액션 생성
for item_name in items.keys():
    actions.append(create_drop_action(item_name))

# 연결된 장소 간 이동 액션 생성
for from_place, to_places in place_connections.items():
    for to_place in to_places:
        # 장소에 따라 비용 정의 (예시로 간단히 설정)
        if (from_place, to_place) in [("sofa", "door"), ("door", "sofa")]:
            time_cost = 0.5
            health_cost = 0.5
        elif (from_place, to_place) in [("sofa", "table"), ("table", "sofa")]:
            time_cost = 0.5
            health_cost = 0.5
        else:
            time_cost = 1.5
            health_cost = 1.5

        actions.append(create_move_action(from_place, to_place, time_cost, health_cost))

# GOAP 실행 함수
def goap(goal_sentences_with_weights, npc_state, world_state):
    goals = []

    for sentence, weight in goal_sentences_with_weights:
        goal = parse_sentence_to_goal(sentence, actions, weight=weight)
        if goal:
            goals.append(goal)

    # 플래닝 실행
    plan_result = plan(goals, npc_state, world_state, actions)

    current_npc_state = npc_state.copy()
    current_world_state = world_state.copy()

    # 결과 출력 및 플랜 실행
    if plan_result:
        print("플랜이 성공적으로 생성되었습니다:")
        total_cost = {"time": 0, "health": 0, "mental": 0}
        for action in plan_result:
            print(f"- {action.name}")
            for k in total_cost.keys():
                total_cost[k] += action.cost.get(k, 0)
            current_npc_state, current_world_state = action.apply(current_npc_state, current_world_state)

        print("\n총 비용:")
        for k, v in total_cost.items():
            print(f"{k}: {v}")
    else:
        print("목표를 달성할 수 없습니다. 가만히 있습니다.")
        plan_result = []
    return plan_result, current_npc_state, current_world_state

# 테스트: 여러 목표 설정
goal_example = [
    ("Do greet", 1),
    ("Do eat_snack", 6),
    #("Go to sofa", 5)
    ("Do sit_sofa", 5)
]

goap(goal_example, npc_state, world_state)


현재 상태: NPCState(upper_body={'hold': 'none'}, lower_body={'location': 'sofa', 'pose': 'stand'}, resources={'time': 0, 'health': 100, 'mental': 100}, inventory=[], state_data={})
달성된 목표: frozenset()
효과 'did_greet': 'True' 확인 결과: False
모든 목표 달성 여부: False
액션 'greet' 적용 후 NPC 상태: NPCState(upper_body={'hold': 'none'}, lower_body={'location': 'sofa', 'pose': 'stand'}, resources={'time': 1, 'health': 99, 'mental': 99}, inventory=[], state_data={'did_greet': True})
액션 'greet' 적용 후 월드 상태: {'table': Place(name=table, inventory=['map', 'snack'], state={}), 'sofa': Place(name=sofa, inventory=['pillow'], state={}), 'door': Place(name=door, inventory=[], state={'door_state': 'closed'})}
효과 'did_greet': 'True' 확인 결과: True
목표 'Do_greet'이(가) 달성되었습니다.
효과 'use_item': 'snack' 확인 결과: False
효과 'pose': 'sit' 확인 결과: False
액션 'sit_sofa' 적용 후 NPC 상태: NPCState(upper_body={'hold': 'none'}, lower_body={'location': 'sofa', 'pose': 'sit'}, resources={'time': 1, 'health': 99, 'mental': 99}, inventory=[], state_data={

([Action(name=move_sofa_to_table, conditions={'pose': 'stand', 'location': 'sofa'}, effects={'location': 'table'}, cost={'time': 0.5, 'health': 0.5, 'mental': 0}),
  Action(name=pick_snack, conditions={'hold': 'none', 'pickup_item': <function create_pick_action.<locals>.<lambda> at 0x7e140af44dc0>}, effects={'hold': 'snack', 'pickup_item': 'snack'}, cost={'time': 0.1, 'health': 0, 'mental': 0}),
  Action(name=eat_snack, conditions={'hold': 'snack'}, effects={'use_item': 'snack', 'hold': 'none', 'used_snack': True}, cost={'time': 0.5, 'health': 0, 'mental': 0}),
  Action(name=move_table_to_sofa, conditions={'pose': 'stand', 'location': 'table'}, effects={'location': 'sofa'}, cost={'time': 0.5, 'health': 0.5, 'mental': 0}),
  Action(name=greet, conditions={'hold': 'none', 'pose': 'stand'}, effects={'did_greet': True}, cost={'time': 1, 'health': 1, 'mental': 1}),
  Action(name=sit_sofa, conditions={'location': 'sofa', 'pose': 'stand'}, effects={'pose': 'sit'}, cost={'time': 1, 'health': 1